# Monthly data generation

In [1]:
import pandas as pd
import numpy as np
import glob
import os

from geopy.distance import geodesic

from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error, mean_squared_error

from scipy.stats import spearmanr


In [2]:
arroz_all = pd.read_parquet('data/prec_monthly_obs_sat.parquet')
arroz_all.head(2)

,latitud,longitud,dpto,mun,station,month_year,fuente,prec_month
0,2.69814,-75.29815,HUILA,CAMPOALEGRE,FEDEARROZ_CAMPOALEGRE_ALTAGRACIA,2011-11,agera5-precipitation,228.52000
1,2.69814,-75.29815,HUILA,CAMPOALEGRE,FEDEARROZ_CAMPOALEGRE_ALTAGRACIA,2011-11,chirps-precipitation,436.49423


In [3]:
# Metricas
def kling_gupta(obs, pred):
    cc = np.corrcoef(obs, pred)[0, 1]  # Correlación de Pearson
    std_obs = np.std(obs)
    std_pred = np.std(pred)
    
    # Evitar división por cero
    if std_obs == 0 or std_pred == 0:
        alpha = np.nan  # Ratio de desviación estándar indefinido
    else:
        alpha = std_pred / std_obs  # Ratio de desviación estándar

    mean_obs = np.mean(obs)
    mean_pred = np.mean(pred)
    
    # Evitar división por cero
    if mean_obs == 0:
        beta = np.nan  # Ratio de medias indefinido
    else:
        beta = mean_pred / mean_obs  # Ratio de medias

    # Calcular KGE con validación de NaN
    if np.isnan(cc) or np.isnan(alpha) or np.isnan(beta):
        return np.nan
    else:
        kge = 1 - np.sqrt((cc - 1)**2 + (alpha - 1)**2 + (beta - 1)**2)
        return kge

# Función para calcular el R^2 ajustado
def r2_ajustado(r2, n, p=1):
    if n <= p + 1:  # Verificación para evitar división por cero o valores negativos
        return np.nan
    else:
        return 1 - ((1 - r2) * (n - 1)) / (n - p - 1)

    
    
# Función para calcular MAPE
def calcular_mape(obs, pred):
    # Evitar división por cero, ignorando los valores donde obs sea cero
    obs_nonzero = obs[obs != 0]
    pred_nonzero = pred[obs != 0]
    return np.mean(np.abs((obs_nonzero - pred_nonzero) / obs_nonzero)) * 100

# Función para calcular MAAPE
def calcular_maape(obs, pred):
    # Evitar división por cero, ignorando los valores donde obs sea cero
    obs_nonzero = obs[obs != 0]
    pred_nonzero = pred[obs != 0]
    return np.mean(np.arctan(np.abs((obs_nonzero - pred_nonzero) / obs_nonzero)))
    
    
def calcular_metricas_monthly(obs, pred):
    if len(obs) < 2 or len(pred) < 2:
        return {
            'r2': None,
            'r2_ajustado': None,
            'rmse': None,
            'mae': None,
            'std_obs': None,
            'std_pred': None,
            'kge': None,
            'spearman': None,
            'bias': None,
            'mape': None,
            'maape': None
        }
    else:
        r2 = r2_score(obs, pred)
        n = len(obs)
        return {
            'r2': r2,
            'r2_ajustado': r2_ajustado(r2, n),
            'rmse': np.sqrt(mean_squared_error(obs, pred)),
            'mae': mean_absolute_error(obs, pred),
            'std_obs': np.std(obs),
            'std_pred': np.std(pred),
            'kge': kling_gupta(obs, pred),
            'spearman': spearmanr(obs, pred).correlation if not np.all(np.isnan(obs) | np.isnan(pred)) else np.nan,
            'bias': np.mean(pred) - np.mean(obs),
            'mape': calcular_mape(obs, pred),
            'maape': calcular_maape(obs, pred)
        }

# Ahora en el bucle que itera por cada estación y mes:
# Aquí solo añado las métricas 'mape' y 'maape' a las que ya tienes en el código original
metricas_mensuales = []

# Iterar por cada estación y cada mes del año (enero, febrero, etc.)
for estacion in arroz_all['station'].unique():
    for mes in range(1, 13): 
        df_filtrado = arroz_all[(arroz_all['station'] == estacion) & (arroz_all['month_year'].dt.month == mes)]

        # Pivotar los datos para tener las fuentes en columnas separadas
        df_pivot = df_filtrado.pivot(index=['latitud', 'longitud', 'station', 'month_year'], 
                                     columns='fuente', 
                                     values='prec_month').dropna()

        # Verificar si tenemos las tres fuentes presentes
        if all(fuente in df_pivot.columns for fuente in ['ideam', 'agera5-precipitation', 'chirps-precipitation']):
            # Calcular las métricas comparando cada fuente con la observada (osb)
            metricas_agera = calcular_metricas_monthly(df_pivot['ideam'], df_pivot['agera5-precipitation'])
            metricas_chirps = calcular_metricas_monthly(df_pivot['ideam'], df_pivot['chirps-precipitation'])
            
            # Guardar los resultados
            metricas_mensuales.append({
                'departamento': arroz_all[(arroz_all['station'] == estacion)]['dpto'].unique()[0],
                'municipio': arroz_all[(arroz_all['station'] == estacion)]['mun'].unique()[0],
                'station': estacion,
                'mes': mes,
                'r2_agera': metricas_agera['r2'],
                'r2_ajustado_agera': metricas_agera['r2_ajustado'],
                'rmse_agera': metricas_agera['rmse'],
                'mae_agera': metricas_agera['mae'],
                'std_obs_agera': metricas_agera['std_obs'],
                'std_pred_agera': metricas_agera['std_pred'],
                'kge_agera': metricas_agera['kge'],
                'spearman_agera': metricas_agera['spearman'],
                'bias_agera': metricas_agera['bias'],
                'mape_agera': metricas_agera['mape'],
                'maape_agera': metricas_agera['maape'],
                'r2_chirps': metricas_chirps['r2'],
                'r2_ajustado_chirps': metricas_chirps['r2_ajustado'],
                'rmse_chirps': metricas_chirps['rmse'],
                'mae_chirps': metricas_chirps['mae'],
                'std_obs_chirps': metricas_chirps['std_obs'],
                'std_pred_chirps': metricas_chirps['std_pred'],
                'kge_chirps': metricas_chirps['kge'],
                'spearman_chirps': metricas_chirps['spearman'],
                'bias_chirps': metricas_chirps['bias'],
                'mape_chirps': metricas_chirps['mape'],
                'maape_chirps': metricas_chirps['maape']
            })

# Convertir los resultados en un dataframe para visualizarlos
df_metricas_mensuales = pd.DataFrame(metricas_mensuales)
df_metricas_mensuales.head(2)

/opt/conda/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/opt/conda/lib/python3.10/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/var/tmp/ipykernel_8484/2024965875.py:79: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  'spearman': spearmanr(obs, pred).correlation if not np.all(np.isnan(obs) | np.isnan(pred)) else np.nan,


,departamento,municipio,station,mes,r2_agera,r2_ajustado_agera,rmse_agera,mae_agera,std_obs_agera,std_pred_agera,...,r2_ajustado_chirps,rmse_chirps,mae_chirps,std_obs_chirps,std_pred_chirps,kge_chirps,spearman_chirps,bias_chirps,mape_chirps,maape_chirps
0,HUILA,CAMPOALEGRE,FEDEARROZ_CAMPOALEGRE_ALTAGRACIA,1,-0.432291,-0.611327,104.202734,85.149,87.068998,61.649527,...,0.533670,56.057519,46.175012,87.068998,64.111181,0.647108,0.357576,-2.503003,35.470575,0.316023
1,HUILA,CAMPOALEGRE,FEDEARROZ_CAMPOALEGRE_ALTAGRACIA,2,-0.219753,-0.372223,67.194785,59.931,60.841450,42.888447,...,0.469703,41.771757,37.004905,60.841450,62.864826,0.737772,0.745455,19.365895,43.839146,0.350530


In [4]:
def format_dataframe(df):
    #  agera
    df_agera = df[['departamento', 'municipio', 'station', 'mes', 'r2_agera', 'r2_ajustado_agera', 'rmse_agera', 'mae_agera', 
                   'std_obs_agera', 'std_pred_agera', 'kge_agera', 'spearman_agera', 'bias_agera','mape_agera','maape_agera']].copy()
    df_agera['source'] = 'agera5'
    df_agera.columns = ['departamento', 'municipio','station', 'mes', 'r2', 'r2_ajustado', 'rmse', 'mae', 'std_obs', 'std_pred', 'kge', 'spearman', 'bias','mape','maape', 'source']
    
    #  chirps
    df_chirps = df[['departamento', 'municipio','station', 'mes', 'r2_chirps', 'r2_ajustado_chirps', 'rmse_chirps', 'mae_chirps', 
                    'std_obs_chirps', 'std_pred_chirps', 'kge_chirps', 'spearman_chirps', 'bias_chirps','mape_chirps','maape_chirps']].copy()
    df_chirps['source'] = 'chirps'
    df_chirps.columns = ['departamento', 'municipio','station', 'mes', 'r2', 'r2_ajustado', 'rmse', 'mae', 'std_obs', 'std_pred', 'kge', 'spearman', 'bias', 'mape','maape','source']
    
    # Concatenar ambos dataframes
    df_format = pd.concat([df_agera, df_chirps], ignore_index=True)
    
    return df_format

# Reestructurar el dataframe
df_metricas_mensuales = format_dataframe(df_metricas_mensuales)
df_metricas_mensuales.head()

,departamento,municipio,station,mes,r2,r2_ajustado,rmse,mae,std_obs,std_pred,kge,spearman,bias,mape,maape,source
0,HUILA,CAMPOALEGRE,FEDEARROZ_CAMPOALEGRE_ALTAGRACIA,1,-0.432291,-0.611327,104.202734,85.149000,87.068998,61.649527,0.082316,-0.066667,-38.045000,50.349702,0.446402,agera5
1,HUILA,CAMPOALEGRE,FEDEARROZ_CAMPOALEGRE_ALTAGRACIA,2,-0.219753,-0.372223,67.194785,59.931000,60.841450,42.888447,0.222341,0.163636,-26.109000,69.225511,0.504275,agera5
2,HUILA,CAMPOALEGRE,FEDEARROZ_CAMPOALEGRE_ALTAGRACIA,3,0.024200,-0.084222,113.514574,96.960000,114.913563,76.125412,0.267899,0.527273,4.892728,76.020062,0.540922,agera5
3,HUILA,CAMPOALEGRE,FEDEARROZ_CAMPOALEGRE_ALTAGRACIA,4,-0.203629,-0.337365,74.644062,48.032727,68.037605,65.202683,0.428948,0.518182,34.945454,229.735901,0.420815,agera5
4,HUILA,CAMPOALEGRE,FEDEARROZ_CAMPOALEGRE_ALTAGRACIA,5,-0.091494,-0.200644,584.099227,249.586666,559.082449,54.429757,-0.616465,0.020979,-91.495000,268.901403,0.916252,agera5


In [5]:
len(df_metricas_mensuales)

912

In [6]:
len(df_metricas_mensuales[df_metricas_mensuales.source=='agera5'])

456

# método de selección de métricas

| Métrica        | Rango                | Mejor valor   | Interpretación                         |
|----------------|----------------------|---------------|-----------------------------------------------|
| **Bias**       | (-∞, ∞)              | 0             | Positivo = sobreestimación, negativo = subestimación |
| **KGE**        | (-∞, 1]              | 1             | 1 es perfecto, negativo indica mal desempeño  |
| **MAAPE**      | [0, 1.57]            | 0             | Más bajo es mejor, cercano a 1.57 indica peores errores |
| **MAPE**       | [0, ∞)               | 0             | Porcentajes más bajos son mejores             |
| **R² ajustado**| (-∞, 1]              | 1             | Explica la variabilidad de los datos, negativo indica mal desempeño |
| **RMSE**       | [0, ∞)               | 0             | Cuanto más bajo, mejor                        |
| **Spearman**   | [-1, 1]              | 1             | Relación monótona perfecta es 1               |


In [15]:
kge= 2
bias=2
maape=2
spearman=1
rmse=1
r=1
def seleccionar_mejores_metricas(df):
    chirps = df[df['source'] == 'chirps']
    agera = df[df['source'] == 'agera5']
    
    puntuacion_chirps = 0
    puntuacion_agera = 0
    
    #  KGE, tener cuidado con el abs
    if np.abs(chirps['kge'].values[0]-1) < np.abs(agera['kge'].values[0]-1):
        puntuacion_chirps += kge
    else:
        puntuacion_agera += kge
    
    #  Bias 0
    if np.abs(chirps['bias'].values[0]) < np.abs(agera['bias'].values[0]):
        puntuacion_chirps += bias
    else:
        puntuacion_agera += bias
    
    #  MAAPE 0
    if np.abs(chirps['maape'].values[0]) < np.abs(agera['maape'].values[0]):
        puntuacion_chirps += maape
    else:
        puntuacion_agera += maape
    
    # Spearman 1 (corro de 2 a cero)
    if np.abs(chirps['spearman'].values[0] - 1) < np.abs(agera['spearman'].values[0] - 1):
        puntuacion_chirps += spearman
    else:
        puntuacion_agera += spearman
    
    # RMSE chiqui
    if chirps['rmse'].values[0] < agera['rmse'].values[0]:
        puntuacion_chirps += rmse
    else:
        puntuacion_agera += rmse
    
    #  R² ajustado 1, tener cuidado con el -
    if np.abs(chirps['r2_ajustado'].values[0] - 1) < np.abs(agera['r2_ajustado'].values[0] - 1):
        puntuacion_chirps += r
    else:
        puntuacion_agera += r

    # Determinar la fuente ganadora
    fuente_mejor = 'chirps' if puntuacion_chirps > puntuacion_agera else 'agera'
    
    # Seleccionar las métricas de la fuente ganadora
    if fuente_mejor == 'chirps':
        metrics = chirps.drop(columns=['source']).squeeze()
    else:
        metrics = agera.drop(columns=['source']).squeeze()

    # Añadir la fuente ganadora a las métricas
    metrics['source_seleccionada'] = fuente_mejor
    
    return metrics


df_metricas_mejores = df_metricas_mensuales.groupby(['departamento', 'municipio', 'station', 'mes']).apply(seleccionar_mejores_metricas)

# Evitar la duplicación de 'mes' verificando si ya existe como columna antes de resetear el índice
if 'mes' in df_metricas_mejores.columns:
    df_metricas_mejores = df_metricas_mejores.reset_index(drop=True)
else:
    df_metricas_mejores = df_metricas_mejores.reset_index()

# Mostrar los primeros resultados
df_metricas_mejores


/var/tmp/ipykernel_8484/3502593289.py:65: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_metricas_mejores = df_metricas_mensuales.groupby(['departamento', 'municipio', 'station', 'mes']).apply(seleccionar_mejores_metricas)


,departamento,municipio,station,mes,r2,r2_ajustado,rmse,mae,std_obs,std_pred,kge,spearman,bias,mape,maape,source_seleccionada
0,ANTIOQUIA,NECHI,FEDEARROZ_NECHI_SANTA_CLARA,1,-4.879833,-6.349791,39.853394,33.773333,16.435497,19.580390,-1.272949,0.600000,33.773333,412.871838,1.063473,agera
1,ANTIOQUIA,NECHI,FEDEARROZ_NECHI_SANTA_CLARA,2,-3.736987,-4.684384,26.705239,21.524123,12.270023,25.438701,-0.438444,0.642857,16.438727,111.608553,0.724336,chirps
2,ANTIOQUIA,NECHI,FEDEARROZ_NECHI_SANTA_CLARA,3,0.109126,-0.069049,47.482934,39.558883,50.307106,64.737882,0.436166,0.750000,30.423597,192.403217,0.681166,chirps
3,ANTIOQUIA,NECHI,FEDEARROZ_NECHI_SANTA_CLARA,4,-0.845870,-1.153515,99.853776,92.426126,73.496037,67.641347,0.232357,0.642857,92.426126,130.312710,0.706510,chirps
4,ANTIOQUIA,NECHI,FEDEARROZ_NECHI_SANTA_CLARA,5,0.224916,0.069899,78.334369,64.395714,88.977025,55.101569,0.366533,-0.035714,5.292857,27.419916,0.256166,agera
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,TOLIMA,VENADILLO,FEDEARROZ_VENADILLO_LA_SONORA,8,0.284276,0.141132,55.522385,43.815677,65.628950,40.069453,0.411158,0.535714,20.484777,115.692186,0.569973,chirps
452,TOLIMA,VENADILLO,FEDEARROZ_VENADILLO_LA_SONORA,9,-1.280494,-1.533882,67.189062,55.236364,44.492221,13.720040,-0.238163,0.172727,-51.536364,78.765477,0.648107,agera
453,TOLIMA,VENADILLO,FEDEARROZ_VENADILLO_LA_SONORA,10,-1.501242,-1.728628,98.929831,76.550000,62.553181,46.596147,-0.134411,-0.027473,-59.946923,50.308154,0.449235,agera
454,TOLIMA,VENADILLO,FEDEARROZ_VENADILLO_LA_SONORA,11,-0.748696,-0.923566,73.171951,51.017500,55.333413,26.514936,-0.299637,-0.132867,-35.905833,69.399486,0.510160,agera


In [16]:
len(df_metricas_mejores[df_metricas_mejores.source_seleccionada=='agera'])*100/len(df_metricas_mejores)

33.55263157894737

In [1]:
df_metricas_mejores[df_metricas_mejores.source_seleccionada=='agera']['departamento'].value_counts()

NameError: name 'df_metricas_mejores' is not defined

In [ ]:

meses_nombres = {1: 'enero', 2: 'febrero', 3: 'marzo', 4: 'abril', 5: 'mayo', 6: 'junio',
                 7: 'julio', 8: 'agosto', 9: 'septiembre', 10: 'octubre', 11: 'noviembre', 12: 'diciembre'}

# Asegurarse de que las columnas 'mes' y 'source' sean categóricas
df_metricas_mensuales['mes'] = df_metricas_mensuales['mes'].astype('category')
df_metricas_mensuales['source'] = df_metricas_mensuales['source'].astype('category')

# Crear la tabla pivote
df_pivot = df_metricas_mensuales.pivot_table(
    index=[ 'departamento','municipio', 'station'],  # Índices
    columns=['mes', 'source'],  # Niveles de columnas
    values=[ 'r2_ajustado', 'rmse',  'kge', 'spearman', 'bias', 'mape', 'maape'],  # Métricas
    aggfunc='first'  # Usamos 'first' porque no queremos agregación
)

# Reorganizar para tener las métricas como el nivel más profundo (nivel 3)
df_pivot.columns = df_pivot.columns.reorder_levels([1, 2, 0])

# Cambiar los nombres de los meses
df_pivot.rename(columns=meses_nombres, level=0, inplace=True)

# Ordenar correctamente los niveles de columnas
df_pivot = df_pivot.sort_index(axis=1)

# Mostrar el resultado
df_pivot.head()


In [ ]:
df_pivot.to_excel('metricas_todas.xlsx')

In [ ]:
columns_nivel_1 = 'mes'      # Primer nivel: Meses
columns_nivel_2 = 'source'   # Segundo nivel: Fuente (chirps, agera)
columns_nivel_3 = ['r2', 'r2_ajustado', 'rmse', 'mae', 'std_obs', 'std_pred', 'kge', 'spearman', 'bias']  # Tercer nivel: Métricas

# Pivotar el dataframe para crear los niveles de columnas
df_pivot = df_metricas_mensuales.pivot_table(
    index=['departamento', 'municipio', 'station'],  # Mantener estas columnas como índice
    columns=[columns_nivel_1, columns_nivel_2],      # Columnas multinivel: mes y source
    values=columns_nivel_3                          # Las métricas dentro de cada fuente y mes
)

# Opcional: renombrar columnas (si es necesario) y ordenar las columnas
df_pivot.columns = pd.MultiIndex.from_tuples(df_pivot.columns, names=['Mes', 'Source', 'Métrica'])

# Restablecer el índice para convertirlo nuevamente en un dataframe
df_pivot = df_pivot.reset_index()
df_pivot

In [ ]:
df_pivot = df_metricas_mensuales.pivot_table(
    index=['departamento', 'municipio', 'station'],  # Mantener estos como índice
    columns=[ 'mes'],  # Nivel 1: source, Nivel 2: meses
    values=['r2', 'r2_ajustado', 'rmse', 'mae', 'kge', 'spearman']  # Nivel 3: métricas
)

# Reorganizamos las columnas para que los meses estén antes de las fuentes
df_pivot = df_pivot.swaplevel(0, 1, axis=1)
df_pivot


In [ ]:
#df_metricas_mensuales.to_parquet('data/metricas_mensuales.parquet')

In [ ]:
def obtener_trimestre(mes):
    if mes in [1, 2, 3]:
        return 'EFM'
    elif mes in [4, 5, 6]:
        return 'AMJ'
    elif mes in [7, 8, 9]:
        return 'JAS'
    elif mes in [10, 11, 12]:
        return 'OND'

# Lista para almacenar los resultados
metricas_3m = []

# Iterar por cada estación y cada trimestre
for estacion in arroz_all['station'].unique():
    for trimestre, meses in {'EFM': [1, 2, 3],
                             'AMJ': [4, 5, 6],
                             'JAS': [7, 8, 9],
                             'OND': [10, 11, 12]}.items():
        
        # Filtrar el dataframe por estación y trimestre
        df_filtrado = arroz_all[(arroz_all['station'] == estacion) & (arroz_all['month_year'].dt.month.isin(meses))]

        # Pivotar los datos para tener las fuentes en columnas separadas
        df_pivot = df_filtrado.pivot(index=['latitud', 'longitud', 'station', 'month_year'], 
                                     columns='fuente', 
                                     values='prec_month').dropna()

        # Verificar si tenemos las tres fuentes presentes
        if all(fuente in df_pivot.columns for fuente in ['ideam', 'agera5-precipitation', 'chirps-precipitation']):
            # Calcular las métricas comparando cada fuente con la observada (osb)
            metricas_agera = calcular_metricas_monthly(df_pivot['ideam'], df_pivot['agera5-precipitation'])
            metricas_chirps = calcular_metricas_monthly(df_pivot['ideam'], df_pivot['chirps-precipitation'])
            
            # Guardar los resultados
            metricas_3m.append({
                'departamento':arroz_all[(arroz_all['station'] == estacion)]['dpto'].unique()[0],
                'municipio': arroz_all[(arroz_all['station'] == estacion)]['mun'].unique()[0],
                'station': estacion,
                'trimestre': trimestre,
                'r2_agera': metricas_agera['r2'],
                'r2_ajustado_agera': metricas_agera['r2_ajustado'],
                'rmse_agera': metricas_agera['rmse'],
                'mae_agera': metricas_agera['mae'],
                'std_obs_agera': metricas_agera['std_obs'],
                'std_pred_agera': metricas_agera['std_pred'],
                'kge_agera': metricas_agera['kge'],
                'spearman_agera': metricas_agera['spearman'],
                'bias_agera': metricas_agera['bias'],
                'r2_chirps': metricas_chirps['r2'],
                'r2_ajustado_chirps': metricas_chirps['r2_ajustado'],
                'rmse_chirps': metricas_chirps['rmse'],
                'mae_chirps': metricas_chirps['mae'],
                'std_obs_chirps': metricas_chirps['std_obs'],
                'std_pred_chirps': metricas_chirps['std_pred'],
                'kge_chirps': metricas_chirps['kge'],
                'spearman_chirps': metricas_chirps['spearman'],
                'bias_chirps': metricas_chirps['bias']
            })

# Convertir los resultados en un dataframe para visualizarlos
df_metricas_3m = pd.DataFrame(metricas_3m)

In [ ]:
def format_dataframe_3m(df):
    #  agera
    df_agera = df[['departamento', 'municipio','station', 'trimestre', 'r2_agera', 'r2_ajustado_agera', 'rmse_agera', 'mae_agera', 
                   'std_obs_agera', 'std_pred_agera', 'kge_agera', 'spearman_agera', 'bias_agera']].copy()
    df_agera['source'] = 'agera5'
    df_agera.columns = ['departamento', 'municipio','station', 'trimestre', 'r2', 'r2_ajustado', 'rmse', 'mae', 'std_obs', 'std_pred', 'kge', 'spearman', 'bias', 'source']
    
    #  chirps
    df_chirps = df[['departamento', 'municipio','station', 'trimestre', 'r2_chirps', 'r2_ajustado_chirps', 'rmse_chirps', 'mae_chirps', 
                    'std_obs_chirps', 'std_pred_chirps', 'kge_chirps', 'spearman_chirps', 'bias_chirps']].copy()
    df_chirps['source'] = 'chirps'
    df_chirps.columns = ['departamento', 'municipio','station', 'trimestre', 'r2', 'r2_ajustado', 'rmse', 'mae', 'std_obs', 'std_pred', 'kge', 'spearman', 'bias', 'source']
    
    # Concatenar ambos dataframes
    df_format = pd.concat([df_agera, df_chirps], ignore_index=True)
    
    return df_format

# Reestructurar el dataframe
df_metricas_3m = format_dataframe_3m(df_metricas_3m)
df_metricas_3m.head()

In [ ]:
#df_metricas_3m.to_parquet('data/metricas_3m.parquet')